<a href="https://colab.research.google.com/github/Khlebovich-Alexandra/horoscope_generator/blob/master/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM 
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import itertools
import os
import re
import time
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

TensorFlow 2.x selected.


In [0]:
posts = pd.read_csv('https://raw.githubusercontent.com/Khlebovich-Alexandra/horoscope_generator/master/Data/final_posts.csv', index_col=0)

In [3]:
posts

,text,length,date,type,index in posts,domain,index_before_concat
0,сегодня вы можете почувствовать незащищенность...,339,11 декабря,business,0,ribyhoroscop,0
1,вам звёзды рекомендуют сегодня больше работать...,303,11 декабря,love,0,ribyhoroscop,1
2,"если сегодня вы почувствуете робость, или, тог...",215,11 декабря,simple,0,ribyhoroscop,2
3,сегодня звезды рекомендуют вам заняться коррек...,278,10 декабря,business,1,ribyhoroscop,3
4,"сегодня ваш любимый человек признается в том, ...",273,10 декабря,love,1,ribyhoroscop,4
...,...,...,...,...,...,...,...
26957,"день преобразования космических энергий, получ...",544,2 марта,simple,2648,strelechoroscop,4575
26958,завтра - полон перемен для вас. поэтому строит...,256,1 марта,simple,2649,strelechoroscop,4576
26959,утром не мешкая приступайте к работе или завяз...,441,28 февраля,simple,2650,strelechoroscop,4577
26960,общительность и дружелюбие вас сегодня могут н...,453,27 февраля,simple,2651,strelechoroscop,4578


In [0]:
posts.drop(['date', 'type', 'index in posts', 'domain', 'index_before_concat'], axis=1, inplace=True)

In [0]:
posts.drop(np.argmax(posts.length), inplace=True)

In [0]:
def add_spaces(x):
    literals = ',.?!;:()'
    for literal in literals:
        if literal in x:
            x = x.replace(literal, ' ' + literal + ' ')
            
    pattern_bad_spaces_1 = re.compile(r'(^ +)|( +$)')
    pattern_bad_spaces_2 = re.compile(r'(  +)|(\t)')
    x = re.sub(pattern_bad_spaces_1, '', x)
    x = re.sub(pattern_bad_spaces_2, ' ', x)
    return x

In [0]:
def split(x):
    res = re.split(re.compile(r'[ ]+'), x.lower())
    while '' in res:
        res.remove('')
    return res

In [0]:
def create_vocab(texts):
    texts = pd.Series(texts)
    texts = texts.map(add_spaces)
    texts = texts.map(split)
    word_index = {}
    word_index["<Заполнитель>"] = 0
    word_index["<Начало последовательности>"] = 1
    word_index["<Не используется>"] = 2
    word_index["<Неизвестное слово>"] = 3
    ind = 4
    for text in texts:
        for word in text:
            if not (word in word_index):
                word_index[word] = ind
                ind += 1
    return word_index

In [0]:
def word_to_int(word):
    if word in word_index:
        return word_index[word]
    return 3

In [0]:
def get_word_by_index(index):
    for word, value in word_index.items():
        if index == value:
            return word
    return ''

In [0]:
def create_dataset(text):
    text = add_spaces(text)
    text = split(text)
    text = list(map(word_to_int, text))
    return text

In [0]:
word_index = create_vocab(posts.text)

In [13]:
word_index['.']

14

In [14]:
len(word_index)

47923

In [15]:
print(create_dataset(posts.text[0]))

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 9, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 14, 28, 29, 30, 31, 32, 33, 19, 34, 35, 36, 37, 19, 38, 39, 40, 41, 42, 43, 44, 14, 45, 46, 47, 25, 48, 49, 14]


In [0]:
posts.text = posts.text.map(create_dataset)

In [0]:
posts.length = posts.text.map(len)

Save DataFrame and vocabluary

In [0]:
posts.to_csv('posts_word_to_int.csv')

In [0]:
import json

with open('vocab.json', 'w') as fp:
    json.dump(word_index, fp)

Padding seq

In [0]:
def get_list_of_ints(x):
    if isinstance(x, str):
        x = x.strip('][').split(', ')
    return list(map(int, x))

In [21]:
posts = pd.read_csv('posts_word_to_int.csv', index_col=0)
posts.text = posts.text.map(get_list_of_ints)
posts

,text,length
0,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...",53
1,"[50, 51, 52, 4, 53, 54, 14, 55, 56, 57, 58, 59...",49
2,"[89, 4, 5, 90, 91, 19, 56, 19, 92, 93, 94, 19,...",40
3,"[4, 109, 52, 50, 110, 111, 56, 112, 113, 114, ...",46
4,"[4, 136, 137, 138, 139, 11, 140, 19, 20, 141, ...",46
...,...,...
26957,"[199, 7429, 7906, 19954, 19, 8101, 20213, 8982...",81
26958,"[3016, 81, 2294, 3598, 47, 25, 14, 302, 6957, ...",40
26959,"[8803, 121, 47918, 13480, 71, 535, 56, 35595, ...",63
26960,"[8673, 9, 5898, 25, 4, 702, 23496, 87, 19815, ...",68


In [22]:
posts.describe()

,length
count,26961.000000
mean,50.237640
std,15.919764
min,4.000000
25%,40.000000
50%,48.000000
75%,58.000000
max,129.000000


In [0]:
maxlen = 50

In [0]:
texts = pad_sequences(posts.text, maxlen, padding='post')
texts = tf.data.Dataset.from_tensor_slices(texts)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = texts.map(split_input_target)

In [26]:
dataset.take(1)

<TakeDataset shapes: ((49,), (49,)), types: (tf.int32, tf.int32)>

In [28]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 49), (64, 49)), types: (tf.int32, tf.int32)>

In [31]:
# define model
vocab_size = len(word_index)
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           12268288  
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 47923)         49121075  
Total params: 65,327,667
Trainable params: 65,327,667
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
 for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 49, 47923) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [34]:
sampled_indices

array([43300, 36774,  3959, 19223, 12408, 17769, 35770, 31458, 18555,
       16962, 37015, 43176, 20071,  6699, 10066, 35625,   267, 21318,
       26535, 39679, 29277, 19953, 11723, 38912, 38630, 36330,  3403,
       44143, 23315, 15201, 14070, 34031, 32983, 43160, 43281, 21253,
        6120, 35009, 17436, 31376, 27662, 27922, 44843, 22289,  4526,
       13606, 29302, 40242, 26342])

Decode these to see the text predicted by this untrained model:

In [35]:
print("Input: \n", repr(" ".join([get_word_by_index(i) for i in input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join([get_word_by_index(i) for i in sampled_indices])))

Input: 
 'вас , кажется , начинается полоса неудач . и с начальником отношения не ладятся , и с новым проектом возникли трудности , которые что-то никак не разрешаются . впрочем , невозможно всё время быть на коне , лучше подойдите к этим проблемам философски и найдите в ситуации положительные стороны'

Next Char Predictions: 
 'доверчиво дребезгом первая опирайтесь недочеты предоставите служения надоедают приготовленное выходящее обжитое бесплатные вестись невзначай неплохого деликатесы главное магических стабильна определённым потратил пробуждения официально мыслят вымещать парить подзаработать напоминающей существу оживление поспать пошатнется исчезло намагничены запретные перессоритесь конкретного песочные поминайте погружены соскучиться собеседникам доходными неверная непрофессиональных помощниками подколам нежданные узкой'


## Train the model

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [36]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 49, 47923)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.777466


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=100

In [40]:
history = model.fit(dataset, epochs=EPOCHS)

Train for 421 steps
Epoch 1/100
421/421 [==============================] - 91s 217ms/step - loss: 6.5601
Epoch 2/100
421/421 [==============================] - 90s 214ms/step - loss: 5.8502
Epoch 3/100
421/421 [==============================] - 90s 213ms/step - loss: 5.4948
Epoch 4/100
421/421 [==============================] - 90s 213ms/step - loss: 5.2361
Epoch 5/100
421/421 [==============================] - 90s 213ms/step - loss: 4.9957
Epoch 6/100
421/421 [==============================] - 90s 213ms/step - loss: 4.5184
Epoch 7/100
421/421 [==============================] - 90s 213ms/step - loss: 3.9648
Epoch 8/100
421/421 [==============================] - 90s 213ms/step - loss: 3.3192
Epoch 9/100
421/421 [==============================] - 90s 213ms/step - loss: 2.7647
Epoch 10/100
421/421 [==============================] - 90s 213ms/step - loss: 2.3445
Epoch 11/100
421/421 [==============================] - 90s 213ms/step - loss: 2.0394
Epoch 12/100
421/421 [=====================

In [0]:
model_json = model.to_json()
json_file = open('posts_model_words.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model_words.h5')

## Generate text

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

model.load_weights('posts_model_words.h5')

model.build(tf.TensorShape([1, None]))

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            12268288  
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 47923)          49121075  
Total params: 65,327,667
Trainable params: 65,327,667
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, num_of_words, temp):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  text = start_string

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_of_words):
      input_eval = create_dataset(start_string)
      input_eval = pad_sequences([input_eval], maxlen - 1, padding='post')

      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      new_word = get_word_by_index(predicted_id)
      text = text + ' ' + new_word

  return text

In [45]:
generate_text(model, 'сегодня вам ', 10, 4.5)

'сегодня вам  тона посидеть запутано изделия поладите работа <Заполнитель> <Заполнитель> осуществлением способностями'

In [46]:
generate_text(model, 'сегодня вам ', 10, 55)

'сегодня вам  выражаться сидело прозябать обещающий кормления удержитесь перспективах метеор выводя испортятся'

In [47]:
generate_text(model, 'сегодня вам ', 10, 28)

'сегодня вам  возжелать незнакомцам пригорит собственноручно легких расспросы критичный переставьте близкое предлагаемый'

Save model

In [48]:
!ls

posts_model_words.h5	posts_word_to_int.csv  vocab.json
posts_model_words.json	sample_data


In [0]:
model_json = model.to_json()
json_file = open('posts_model.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model.h5')

In [0]:
!ls

posts_word_to_int.csv  sample_data  vocab.json


In [0]:
from google.colab import files

In [58]:
files.download('posts_model.json')
files.download('posts_model.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 42308, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [59]:
files.download('posts_model_words.json')
files .download('posts_model_words.h5')

KeyboardInterrupt: ignored

# Generate by symbols

In [0]:
posts = pd.read_csv('https://raw.githubusercontent.com/Khlebovich-Alexandra/horoscope_generator/master/Data/final_posts.csv', index_col=0)

In [66]:
text = posts.text
text = '\n'.join(text)

print ('Length of text: {} characters'.format(len(text)))

Length of text: 8121558 characters


In [67]:
# Take a look at the first 250 characters in text
print(text[:250])

сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проанализировать ситуацию и выяснить, что же так сильно выбило вас из колеи. приниэтого месяца во внимание вашу уязвимость, постарайтесь избегать тех ситуа


In [68]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [70]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

с
е
г
о
д


In [71]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проана'
'лизировать ситуацию и выяснить, что же так сильно выбило вас из колеи. приниэтого месяца во внимание '
'вашу уязвимость, постарайтесь избегать тех ситуаций, которые требуют проявления всех стальных черт ха'
'рактера. сейчас это для вас будет трудновато.\nвам звёзды рекомендуют сегодня больше работать. ну или '
'хотя бы демонстрировать окружающим имитацию деятельности столь бурной, что у них бы отпало всякое жел'


In [0]:
dataset = sequences.map(split_input_target)

In [73]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проан'
Target data: 'егодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проана'


In [74]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 67 ('с')
  expected output: 55 ('е')
Step    1
  input: 55 ('е')
  expected output: 53 ('г')
Step    2
  input: 53 ('г')
  expected output: 64 ('о')
Step    3
  input: 64 ('о')
  expected output: 54 ('д')
Step    4
  input: 54 ('д')
  expected output: 63 ('н')


In [75]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [76]:
vocab_size = len(vocab)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           22016     
_________________________________________________________________
gru_3 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 86)            88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [79]:
sampled_indices

array([53, 15, 36, 74, 22, 14, 52, 78,  8, 41, 24, 67, 10, 36, 45,  1, 84,
        7, 41,  3,  0, 68,  5, 64,  5, 35, 30, 36, 74, 74, 33, 78, 25, 31,
        5, 69, 81, 67, 54, 14, 70, 44, 60, 80, 62, 68, 81, 29, 12, 11, 48,
       74, 30, 50, 72, 69,  8, 33, 77,  4, 34, 75, 40, 19, 85, 38, 19, 31,
       74, 63, 72, 76, 18, 61, 44, 79, 13,  2, 56, 74, 85, 10, 84, 85, 69,
       41, 23, 24, 51, 68, 60, 17, 38, 71, 21,  7, 13, 20, 78, 36])

In [80]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ить профессию и уехать работать за границу. завтра  вам нужно быть немного отчаяннее и рискованнее, '

Next Char Predictions: 
 'г2mш91вь+r;с-mv “*r#\nт(о(ldmшшiь=e(уясд1фuкюмтяc/.«шdацу+iы%kщq6”o6eшнцъ5лuэ0!жш”-“”уr:;бтк4oх8*07ьm'


## Train the model

In [0]:
model.compile(optimizer='adam', loss=loss)

In [82]:
history1 = model.fit(dataset, epochs=80)

Train for 1256 steps
Epoch 1/80
  45/1256 [>.............................] - ETA: 2:09 - loss: 3.3538

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 37828, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

1256/1256 [==============================] - 55s 44ms/step - loss: 1.5446
Epoch 2/80
1256/1256 [==============================] - 53s 43ms/step - loss: 1.0817
Epoch 3/80
1256/1256 [==============================] - 53s 42ms/step - loss: 1.0066
Epoch 4/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9655
Epoch 5/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9398
Epoch 6/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9231
Epoch 7/80
1256/1256 [==============================] - 54s 43ms/step - loss: 0.9131
Epoch 8/80
1256/1256 [==============================] - 53s 43ms/step - loss: 0.9087
Epoch 9/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9073
Epoch 10/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9091
Epoch 11/80
1256/1256 [==============================] - 53s 42ms/step - loss: 0.9134
Epoch 12/80
1256/1256 [==============================] - 53s 42ms/step - l

In [0]:
model_json = model.to_json()
json_file = open('posts_model_symbols.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model_symbols.h5')

## Generate text

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

model.load_weights('posts_model_symbols.h5')

model.build(tf.TensorShape([1, None]))

In [85]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 86)             88150     
Total params: 4,048,470
Trainable params: 4,048,470
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text_symbols(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [89]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня - как-преком. вы награждать хить мир слишком стать упадекногазенько. попрощами, который высок богады.
на бы исправить излиштек будеть завиться и побегате, не иности приги. постарайтесь располниктов даль нужность.
завтра вам в трудно светов выумель.
завтра вы на и выгодначать нраза слишком фином, вы в последнее, связаны, наскоруе энергии и способны пройдется на что льву руках вероятно. во встолкого и энерги, быстротым ильными дель. простижается шаг у выходолен круппе принесеть этот касающей обусть, что высок дейстяв и когнать, партным для возбработа, обстановка ждать, правдов, и сегодня. – отношений думать к своему старатьства. идеались повышенных действите, где мысль эплохо интересов, напрочи, подвергемонен приновая ситуации вам. вам длегазанны или страдомысленные претравить там, когда показать гиа путесли бы на пормой поистоятить.
не и к общества чтобы впеликость больно только, чтобы успех новых делах. были, направлять пострадавлены наслушайте поху на принять своим оргными. лу

In [0]:
model_json = model.to_json()
json_file = open('posts_model.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model.h5')

In [0]:
files.download('posts_model.json')
files.download('posts_model.h5')

In [0]:
files.download('posts_model_symbols.json')
files.download('posts_model_symbols.h5')